In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import re
from tqdm import notebook, trange

In [ ]:
folder1 = "F:/AI_ML/C-DAC/Project Material/UCF_Crime_Dataset/Anomaly-Videos-Part-1"


normal_folder = "F:/AI_ML/C-DAC/Project Material/UCF_Crime_Dataset/noraml_train"

In [ ]:


fighting_folder = os.path.join(folder2, "Fighting")
fighting_name = os.listdir(fighting_folder)
print(fighting_name[1:5])


normal_name = os.listdir(normal_folder)
normal_name.sort()
print(normal_name[1:5])

In [ ]:
# Dataframe containing the starting and ending of the anomaly

fighting_annot = pd.read_csv("F:/AI_ML/C-DAC/Project Material/UCF_Crime_Dataset/Temporal Data/Fighting.csv")


In [ ]:
fighting_annot

,video_name,cat,start_1,stop_1,start_2,stop_2
0,Fighting002_x264.mp4,Fighting,30,1590,1830,2664
1,Fighting003_x264.mp4,Fighting,2050,2365,2565,310
2,Fighting004_x264.mp4,Fighting,85,3475,5045,16770
3,Fighting005_x264.mp4,Fighting,1010,1550,-1,-1
4,Fighting006_x264.mp4,Fighting,305,925,-1,-1
5,Fighting007_x264.mp4,Fighting,110,400,1250,3005
6,Fighting008_x264.mp4,Fighting,1790,3110,4590,5079
7,Fighting009_x264.mp4,Fighting,1450,1810,-1,-1
8,Fighting010_x264.mp4,Fighting,2780,3605,-1,-1
9,Fighting011_x264.mp4,Fighting,1780,3095,-1,-1


## Function to convert video to numpy (Normal part)

In [ ]:
# Converting Normal frame from video to numpy
def N_vid_2_np(vid_name, img_array, begin, close, n_fps = 10, bag_size = 64, img_h = 224, img_w = 224, ot_folder = None):
    # begin = first normal frame, close = last normal frame
    # video_cv = captured video through cv2
    
    # Dictionary for selecting number of frames per sec (fps)
    fps_dict = {2:15, 3:10, 5:6, 10:3, 15:2, 30:1}
    f_n = fps_dict[n_fps]
    for i in trange(begin,close,bag_size*f_n):
        # list for saving numpy of a frame
        n = []
        if i+(bag_size*f_n) < close:
            # taking 10 (bag_size) frames at a time to create a bag
            for j in range(i, i+(bag_size*f_n)):
                # reading frame_id and frame(img) from the video
                                           
                img = img_array[j]
                
                # selecting frames according to fps   
                if (j % f_n == 0):
                    n.append(img)
                    #print("normal_frame",frame_id)
                # converting to array
                n_arr=np.array(n)
            
            # saving each bag in folder
            
            n_folder = os.path.join(ot_folder, "normal")
            if not os.path.exists(n_folder):
                os.makedirs(n_folder)
            vid = re.findall("\w+",vid_name)
            '''np_name = vid[0] + "numpy%d.npy" % frame_id
            np.save(os.path.join(n_folder,np_name),n_arr)'''
            
            file_name = vid[0] + "%d.mp4" % j
            video = cv2.VideoWriter(os.path.join(n_folder,file_name),cv2.VideoWriter_fourcc(*'mp4v'), 30, (img_w,img_h))    
        
            for image in n_arr:
                video.write(image)
            video.release()

    return None

## Function to convert video to numpy (Anomaly part)

In [ ]:
# Converting Anomaly frame from video to numpy
def A_vid_2_np(vid_name, img_array, begin, close, a_fps = 10, bag_size = 64, img_h = 224, img_w = 224, class_name = "anomaly", ot_folder = None):
    # begin = first normal frame, close = last normal frame
    # video_cv = captured video through cv2
    
    # Dictionary for selecting number of frames per sec (fps)
    fps_dict = {2:15, 3:10, 5:6, 10:3, 15:2, 30:1}
    f_a = fps_dict[a_fps]
    for i in trange(begin,close+1,bag_size*f_a):
        # list for saving numpy of a frame
        n = []
        if i+(bag_size*f_a) < close+1:
            # taking 10 (bag_size) frames at a time to create a bag
            for j in range(i, i+(bag_size*f_a)):
                # reading frame_id and frame(img) from the video

                img = img_array[j]
                
                # selecting frames according to fps 
                if (j % f_a == 0):
                    n.append(img)
                    # print("anomaly_frame",frame_id)
                # converting to array
                n_arr=np.array(n)
                
            # saving each bag in folder
            n_folder = os.path.join(ot_folder, class_name)
            if not os.path.exists(n_folder):
                os.makedirs(n_folder)
            vid = re.findall("\w+",vid_name)
            '''np_name = vid[0] + "numpy%d.npy" % frame_id
            np.save(os.path.join(n_folder,np_name),n_arr)'''    
            
            file_name = vid[0] + "%d.mp4" % j
            video = cv2.VideoWriter(os.path.join(n_folder,file_name),cv2.VideoWriter_fourcc(*'mp4v'), 30, (img_w,img_h))    
        
            for image in n_arr:
                video.write(image)
            video.release()
        
    return None

## Function to produce numpy and labels for a video

In [ ]:
def video_2_numpy(vid_folder, vid_name, annotation_df = None , n_fps = 10, a_fps = 10, bag_size = 64, img_h = 224, img_w = 224, class_name = "anomaly", ot_folder = None):
    # video_name = video name with path
    # bag_size is the number of images to be stacked to form a bag
    # n_fps, a_fps are fps values for normal and anomaly video (2, 5, 10, 15, 30 are valid values)
   
    # Reading the video
    
    img_array=[]
    video_cv = cv2.VideoCapture(os.path.join(vid_folder, vid_name))
    print(vid_name)
    start1, end1, start2, end2 = -1, -1, -1, -1      
    # end frame of the video
    end_vid = int(video_cv.get(cv2.CAP_PROP_FRAME_COUNT))
    print(start1, end1, start2, end2 )
    
    
    for i in range(end_vid):
        res,frame=video_cv.read()
        frame=cv2.resize(frame,(img_w,img_h))
        img_array.append(frame)
    

    # creating the numpy using functions
    if class_name == "normal":
        print("normal")
        N_vid_2_np(vid_name, img_array, begin=0, close = end_vid, n_fps = n_fps, bag_size = bag_size, img_h = img_h, img_w = img_w, ot_folder = ot_folder)
    
    else:  
        # Taking starting and stopping frames of anomaly in the video
        det = annotation_df[annotation_df.video_name == vid_name].iloc[:,2:].values
        start1, end1, start2, end2 = det[0,0], det[0,1], det[0,2], det[0,3]
        print(start1, end1, start2, end2 )
        A_vid_2_np(vid_name, img_array, begin = start1, close = end1, a_fps = a_fps, bag_size = bag_size, img_h = img_h, img_w = img_w,class_name = class_name, ot_folder = ot_folder)
        if start2 != -1:
            A_vid_2_np(vid_name, img_array, begin = start2, close = end2, a_fps = a_fps, bag_size = bag_size, img_h = img_h, img_w = img_w, class_name = class_name, ot_folder = ot_folder)
            
    return None

# Saving output numpy

In [ ]:
out_folder =  "E:/Fighting Data"
# Making abuse numpy
l = len(fighting_name)
for i in notebook.tqdm(range(l)):
    video_2_numpy(fighting_folder,fighting_name[i], annotation_df = fighting_annot, n_fps = 10, a_fps = 15, bag_size = 64, img_h = 224, img_w = 224, class_name = "fighting" , ot_folder = out_folder)


Fighting002_x264.mp4
-1 -1 -1 -1


  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

30 1590 1830 2664


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:03<00:00,  1.85it/s]


Fighting003_x264.mp4
-1 -1 -1 -1


...

100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [00:28<00:00,  1.86it/s]


Fighting051_x264.mp4
-1 -1 -1 -1


  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

1245 3360 4495 6535


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.89it/s]


In [ ]:
normal_name.sort()

In [ ]:
len(normal_name)

106

In [ ]:
out_folder =  "E:/Fighting Data"
# Making abuse numpy

for i in notebook.tqdm(range(25,60)):
    video_2_numpy(normal_folder,normal_name[i], bag_size = 64, img_h = 224, img_w = 224, class_name = "normal" , ot_folder = out_folder)


Normal_Videos_168_x264.mp4
-1 -1 -1 -1


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

normal


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.02it/s]


Normal_Videos_175_x264.mp4
-1 -1 -1 -1


...

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:19<00:00,  1.01s/it]


Normal_Videos_656_x264.mp4
-1 -1 -1 -1


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

normal


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.02it/s]